# 고급 추론 및 문제 해결 모델 탐색  

OpenAI의 o1 시리즈 모델은 강화 학습을 통해 훈련된 대형 언어 모델로, 복잡한 추론을 수행할 수 있도록 설계되었습니다. o1 모델은 답변하기 전에 깊이 사고하며, 내부적으로 긴 사고 과정을 거친 후 응답을 생성합니다.  

o1 모델은 과학적 추론에서 뛰어난 성능을 보이며, 경쟁 프로그래밍(Codeforces) 문제에서 상위 89퍼센타일을 기록하고, 미국 수학 올림피아드 예선(AIME)에서 미국 상위 500명 학생 수준의 성과를 달성했으며, 물리학, 생물학, 화학 문제를 다루는 GPQA 벤치마크에서 인간 박사급 정답률을 초과했습니다.  

API에서는 두 가지 추론 모델을 사용할 수 있습니다:  

- **o1**: 광범위한 일반 지식을 활용하여 어려운 문제를 추론하도록 설계된 모델  
- **o1-mini**: 보다 빠르고 경제적인 버전의 o1 모델로, 방대한 일반 지식이 필요하지 않은 코딩, 수학, 과학 문제 해결에 특히 강점이 있음  


### Chain of Thought (COT)
**Chain of Thought (COT, 사고의 연결)** 는 복잡한 문제를 해결할 때 단계별로 논리를 전개하며 중간 추론 과정을 명시적으로 표현하는 기법입니다. 이 방법을 사용하면 모델이 더 정확한 결과를 도출할 가능성이 높아집니다.  

- OpenAI의 o1 모델은 Chain of Thought (CoT) 기법을 활용하여 복잡한 문제를 단계별로 추론하는 능력을 강화한 모델입니다. 이 접근 방식은 모델이 응답을 생성하기 전에 더 많은 시간을 들여 '생각'하도록 설계되어, 특히 과학, 수학, 프로그래밍 등에서 향상된 성능을 보입니다. 
- 
o1 모델은 강화 학습을 통해 CoT 방식을 학습하였으며, 이를 통해 문제를 세분화하고 각 단계에서 논리적 추론을 수행합니다. 이러한 단계별 사고 과정은 모델이 복잡한 문제를 해결하는 데 도움을 줍니다. 

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
from openai import OpenAI
client = OpenAI()

Model = "o1-mini"

### 추론 모델 (o1 모델 사용)

In [3]:
prompt = """
문자열로 표현된 행렬(예: '[1,2],[3,4],[5,6]')을 입력으로 받아, 
같은 형식으로 전치 행렬을 출력하는 Python 스크립트를 작성하세요.
"""

response = client.chat.completions.create(
    model=Model,
    messages=[
        {
            "role": "user", 
            "content": prompt
        }
    ]
)

print(response.choices[0].message.content)

아래는 문자열로 표현된 행렬을 입력받아 전치 행렬을 같은 형식으로 출력하는 Python 스크립트 예제입니다. 이 스크립트는 사용자가 입력한 문자열을 파싱하여 행렬로 변환한 후, 전치 행렬을 생성하고 다시 문자열 형식으로 출력합니다.

```python
import ast

def parse_matrix(matrix_str):
    """
    문자열로 표현된 행렬을 파싱하여 리스트의 리스트로 변환합니다.
    예: '[1,2],[3,4],[5,6]' -> [[1, 2], [3, 4], [5, 6]]
    """
    # 문자열을 대괄호로 감싸서 전체를 리스트로 인식하게 함
    wrapped_str = f'[{matrix_str}]'
    try:
        matrix = ast.literal_eval(wrapped_str)
        # 각 행이 리스트인지 확인
        if all(isinstance(row, list) for row in matrix):
            return matrix
        else:
            raise ValueError("모든 행은 리스트여야 합니다.")
    except (SyntaxError, ValueError) as e:
        print("유효한 행렬 형식이 아닙니다.")
        raise e

def transpose_matrix(matrix):
    """
    주어진 행렬의 전치 행렬을 반환합니다.
    """
    # zip(*)을 사용하여 전치
    transposed = list(map(list, zip(*matrix)))
    return transposed

def matrix_to_string(matrix):
    """
    행렬을 문자열 형식으로 변환합니다.
    예: [[1, 3, 5], [2, 4, 6]] -> '[1,3,5],[2,4,6]'
    """
    row_strings = [f'[{",".joi

### 추론 모델의 비용 관리  

o1 시리즈 모델에서 비용을 관리하려면 `max_completion_tokens` 매개변수를 사용하여 모델이 생성하는 총 토큰 수(추론 토큰과 응답 토큰 포함)를 제한할 수 있습니다.  

이전 모델에서는 `max_tokens` 매개변수가 생성되는 토큰 수와 사용자에게 표시되는 토큰 수를 동시에 제어했으며, 이 둘은 항상 동일했습니다. 그러나 o1 시리즈에서는 내부 추론 과정에서 추가 토큰이 생성될 수 있어, 총 생성 토큰 수가 사용자에게 표시되는 토큰 수를 초과할 수 있습니다. 때문에, o1 시리즈에서는 `max_completion_tokens`를 도입하여 모델이 생성하는 총 토큰 수(추론 + 표시된 응답 토큰)를 명확하게 제어할 수 있도록 했습니다. 이를 통해 새로운 모델을 사용할 때 기존 애플리케이션이 정상적으로 작동하도록 보장속 작동합니다.

In [6]:
prompt = """
사용자로부터 질문을 입력받아 데이터베이스에서 해당 질문과 매핑된 답변을 조회하는 Python 애플리케이션을 만들고 싶습니다.  
질문과 유사한 답변이 존재하면 해당 답변을 반환하고, 존재하지 않는 경우 사용자가 답변을 입력하도록 요청한 후  
해당 질문/답변 쌍을 데이터베이스에 저장합니다.  

이 애플리케이션에 필요한 디렉터리 구조를 계획한 후, 각 파일의 전체 내용을 제공하세요.  
코드 중간에 설명을 포함하지 말고, 처음과 끝에서만 이유를 설명하세요.
"""

response = client.chat.completions.create(
    model=Model,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                },
            ],
        }
    ],
    max_completion_tokens=5000
)

print(response.choices[0].message.content)

아래는 요청하신 Python 애플리케이션의 디렉터리 구조와 각 파일의 전체 내용입니다.

```
my_app/
├── main.py
├── db.py
├── similarity.py
├── requirements.txt
└── README.md
```

---

**`main.py`**
```python
import db
import similarity

def main():
    db.initialize()
    while True:
        question = input("질문을 입력하세요 (종료하려면 'exit' 입력): ").strip()
        if question.lower() == 'exit':
            print("프로그램을 종료합니다.")
            break
        answer = db.get_answer(question)
        if answer:
            print(f"답변: {answer}")
        else:
            user_answer = input("답변을 찾을 수 없습니다. 답변을 입력해주세요: ").strip()
            db.add_qa_pair(question, user_answer)
            print("질문과 답변이 저장되었습니다.")

if __name__ == "__main__":
    main()
```

---

**`db.py`**
```python
import sqlite3
from similarity import are_similar

DB_NAME = 'qa.db'
SIMILARITY_THRESHOLD = 0.8

def initialize():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS qa (
            id 

추론 토큰을 위한 충분한 컨텍스트 윈도우 공간 확보가 필요 합니다. 문제의 복잡성에 따라 모델은 수백 개에서 수만 개의 추론 토큰을 생성할 수 있습니다.  

사용된 정확한 추론 토큰 수는 채팅 완성 응답 객체(chat completion response object)의 `usage` 객체 내 `completion_tokens_details`에서 확인할 수 있습니다.

In [7]:
response.usage.to_dict()

{'completion_tokens': 1368,
 'prompt_tokens': 145,
 'total_tokens': 1513,
 'completion_tokens_details': {'audio_tokens': 0,
  'reasoning_tokens': 512,
  'accepted_prediction_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

### COT (Chain of Thought)
**Chain of Thought (CoT)** 는 **대형 언어 모델(LLM)** 이 문제를 해결할 때 논리적 사고 과정을 단계별로 명확하게 표현하도록 유도하는 프롬프트 기법입니다.

일반적인 LLM의 답변 방식은 질문에 대해 즉시 응답하는 것이지만, CoT를 활용하면 모델이 중간 사고 과정을 거쳐 더 정확하고 신뢰할 수 있는 답변을 생성할 수 있습니다. 

##### CoT의 핵심 원리
- 단계별 사고 과정 유도
- 문제를 한 번에 풀도록 요청하는 것이 아니라, **"하나씩 논리적으로 설명한 후 답변을 제시"**하도록 유도하는 방식입니다.
- 모델에게 "Let's think step by step."(단계별로 생각해 보자) 같은 문구를 추가하면 CoT 방식을 유도할 수 있습니다.

### 일반 LLM 모델 (gpt-4o)에 COT 기법 적용

In [13]:
# 테스트할 문제

# question = "자동차가 시속 60km로 2.5시간 동안 이동하면 총 몇 km를 이동하게 될까요?"

# question = """
# 철수는 5개의 골프공을 가지고 있다.
# 오늘 2박스를 더 샀다. 각 박스에는 5개의 골프 공이 들어 있다.
# 이제 철수는 몇 개의 골프 공을 가지고 있는가?
# """

question = """
냉장고에 23개의 사과가 있다. 20개를 먹고 6개를 더 샀다면, 사과는 몇 개가 남아 있는가?
"""

prompt = f"""
    체인 오브 소트(Chain of Thought, CoT) 추론을 사용하여 단계별로 해결해 봅시다.

    질문: {question}

    신중하게 생각하고, 최종 답변을 제공하기 전에 자세한 단계별 설명을 작성하세요.
    """

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
              {
                  "role": "user", 
                  "content": prompt
              }
    ],
    max_completion_tokens=1000
)

print(response.choices[0].message.content)

문제를 단계별로 분석해 보겠습니다.

1. **현재 사과의 수**: 냉장고에 처음에 23개의 사과가 있습니다.
   
2. **먹은 사과의 수**: 이 중에서 20개의 사과를 먹습니다. 그러므로, 먹은 사과의 수를 처음 사과 수에서 빼야 합니다.
   - 23 - 20 = 3
   - 현재 냉장고에는 3개의 사과가 남아 있습니다.

3. **추가한 사과의 수**: 이후, 6개의 사과를 더 삽니다. 기존의 사과 수에 추가한 수를 더해야 합니다.
   - 3 + 6 = 9
   - 최종적으로 냉장고에는 9개의 사과가 남아 있습니다.

따라서, 사과는 최종적으로 **9개**가 남아 있습니다.


<br>

### 동일한 질문을 추론 모델(o1) 이용

In [17]:
response = client.chat.completions.create(
    model=Model,
    messages=[
        {
            "role": "user", 
            "content": question
        }
    ]
)

print(response.choices[0].message.content)

냉장고에 처음에 사과가 23개 있습니다.

1. **사과를 20개 먹음:**
   \[
   23 - 20 = 3 \text{개}
   \]

2. **사과를 6개 더 구매함:**
   \[
   3 + 6 = 9 \text{개}
   \]

따라서, 사과는 **총 9개** 남아 있습니다.
